# Pycosat implementation of Algo #1

In [20]:
import pycosat

In [21]:
# Solver for Numberlink
class NumberLinkSolver:
    pass

In [22]:
# choose variables
# we need a number per edge
# each edge is ((x1, y1), (x2, y2))
# x_{edge, index of path}
from typing import Optional

def x(self, edge, index):
    def edge_to_num(edge):
        v, w = edge
        x1, y1 = v
        x2, y2 = w
        return x1 * self.n**3 + y1 * self.n**2 + x2 * self.n + y2

    num_paths = self.num_paths
    ans = edge_to_num(edge) * num_paths + index
    # +1 because we start from
    return ans + 1

NumberLinkSolver.x = x

def inverse_x(self, var):
    n = self.n
    num_paths = self.num_paths

    var -= 1
    e, index = divmod(var, num_paths)
    rest, y2 = divmod(e, n)
    rest, x2 = divmod(rest, n)
    x1, y1 = divmod(rest, n)

    return ((x1, y1), (x2, y2)), index

NumberLinkSolver.inverse_x = inverse_x

In [23]:
# HELPER METHODS
def all(literals):
    return [[literal] for literal in literals]


def none(literals):
    return [[-literal] for literal in literals]


def at_least_one(literals):
    return [literals]


def at_most_one(literals):
    return [[-l1, -l2] for l1 in literals for l2 in literals if l1 < l2]


def exactly_one(literals):
    return at_least_one(literals) + at_most_one(literals)

In [24]:
def at_most_one_path_per_edge(self):
    num_paths = self.num_paths
    edges = self.edges
    x = self.x

    clauses = []
    for e in edges:
        literals = [x(e, i) for i in range(num_paths)]
        clauses += at_most_one(literals)
    return clauses

NumberLinkSolver.at_most_one_path_per_edge = at_most_one_path_per_edge

In [25]:
def exactly_one_edge_per_vertex_on_path(self):
    num_paths = self.num_paths
    pairs = self.pairs
    x = self.x
    vertex_to_edges = self.vertex_to_edges

    clauses = []
    for index in range(num_paths):
        vertices = pairs[index]
        for v in vertices:
            literals = [x(e, index) for e in vertex_to_edges[v]]
            clauses += exactly_one(literals)

            # don't want these vertices to be on any other non-index paths
            for i in range(num_paths):
                if i != index:
                    literals = [x(e, i) for e in vertex_to_edges[v]]
                    clauses += none(literals)
    return clauses

NumberLinkSolver.exactly_one_edge_per_vertex_on_path = (
    exactly_one_edge_per_vertex_on_path
)

In [26]:
def exactly_two_edges_per_vertex_not_in_pair(self):
    num_paths = self.num_paths
    pairs = self.pairs
    x = self.x
    vertex_to_edges = self.vertex_to_edges
    vertices = self.vertices

    pair_vertices = {v for pair in pairs for v in pair}
    clauses = []
    for v in vertices:
        if v in pair_vertices:
            continue

        # require at least one across ALL paths
        literals = [x(e, i) for e in vertex_to_edges[v] for i in range(num_paths)]
        clauses += at_least_one(literals)

        # for each vertex, we require at least two edges among all paths to be active
        for index in range(num_paths):
            literals = [x(e, index) for e in vertex_to_edges[v]]

            clause = at_least_one(literals)[0]
            for literal in literals:
                clause_copy = clause.copy()
                clause_copy.remove(literal)
                clause_copy.append(-literal) # if literal is True, one of the others must be True
                clauses.append(clause_copy)

    return clauses

NumberLinkSolver.exactly_two_edges_per_vertex_not_in_pair = (
    exactly_two_edges_per_vertex_not_in_pair
)

In [27]:
from collections import defaultdict

def __init__(self, puzzle):
    # puzzle is a list of lists of integers where 0 is empty, and any other number is a vertex
    n = len(puzzle)
    pairs = []
    pair_matches = defaultdict(list)
    for x in range(n):
        for y in range(n):
            vertex = puzzle[x][y]
            if vertex != 0:
                pair_matches[vertex].append((x, y))

    for pair in pair_matches.values():
        pairs.append(pair)

    self.n = n
    self.pairs = pairs
    self.num_paths = len(pairs)
    self.vertices = [(x, y) for x in range(n) for y in range(n)]

    edges = []
    vertex_to_edges = defaultdict(list)
    for x in range(n):
        for y in range(n):
            if x + 1 < n:
                edges.append(((x, y), (x + 1, y)))
                vertex_to_edges[(x, y)].append(((x, y), (x + 1, y)))
                vertex_to_edges[(x + 1, y)].append(((x, y), (x + 1, y)))
            if y + 1 < n:
                edges.append(((x, y), (x, y + 1)))
                vertex_to_edges[(x, y)].append(((x, y), (x, y + 1)))
                vertex_to_edges[(x, y + 1)].append(((x, y), (x, y + 1)))

    self.edges = edges
    self.vertex_to_edges = vertex_to_edges


NumberLinkSolver.__init__ = __init__

In [28]:
def solve(self):
    clauses = []
    clauses += self.at_most_one_path_per_edge()
    clauses += self.exactly_one_edge_per_vertex_on_path()
    clauses += self.exactly_two_edges_per_vertex_not_in_pair()

    literals = pycosat.solve(clauses)

    if literals == "UNSAT":
        return None

    n = self.n
    solution = [[0 for _ in range(n)] for _ in range(n)]
    for literal in literals:
        if literal > 0:
            edge, index = self.inverse_x(literal)
            x1, y1 = edge[0]
            x2, y2 = edge[1]
            solution[x1][y1] = index + 1
            solution[x2][y2] = index + 1
    return solution


NumberLinkSolver.solve = solve

In [29]:
puzzle = [[1, 0, 1], [2, 0, 2], [3, 0, 3]]

solver = NumberLinkSolver(puzzle)
print(solver.solve())  # [[1, 1, 1], [2, 2, 2], [3, 3, 3]]

[[1, 1, 1], [2, 2, 2], [3, 3, 3]]
